<a href="https://colab.research.google.com/github/AbdulrahmanAlshugaa/AbdulrahmanAlshugaa/blob/main/project_Arabic_Words_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#================Cell 1==================
Download and decompress the form full_grams_cbow_300_twitter.zip that was trained on data previously in order to be available here during the code execution

In [ ]:
!wget https://archive.org/download/full_grams_cbow_300_twitter/full_grams_cbow_300_twitter.zip
!unzip full_grams_cbow_300_twitter.zip

--2022-03-13 09:16:41--  https://archive.org/download/full_grams_cbow_300_twitter/full_grams_cbow_300_twitter.zip
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia802803.us.archive.org/15/items/full_grams_cbow_300_twitter/full_grams_cbow_300_twitter.zip [following]
--2022-03-13 09:16:42--  https://ia802803.us.archive.org/15/items/full_grams_cbow_300_twitter/full_grams_cbow_300_twitter.zip
Resolving ia802803.us.archive.org (ia802803.us.archive.org)... 207.241.232.113
Connecting to ia802803.us.archive.org (ia802803.us.archive.org)|207.241.232.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3325529808 (3.1G) [application/zip]
Saving to: ‘full_grams_cbow_300_twitter.zip’

             full_g   0%[                    ]  21.04M   363KB/s    eta 66m 38s

#=============Cell 2==============
This cell to call the necessary offices and the forms that will be required to fetch in order to execute the code correctly

In [ ]:
import gensim
import re
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 



#===============Cell 3=================
Creat Methods:
1- The function(Clean_and_Norm_Arabic_Text) is used to clean and normalize Arabic texts and remove distractions from them
2- The function(get_vector) is used to find the vector to be extracted from the tokens entered into the model

In [ ]:

def Clean_and_Norm_Arabic_Text(Arbic_text):
    List_replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    List_search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
  
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    Arbic_text = re.sub(p_tashkeel,"", Arbic_text)

    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    Arbic_text = re.sub(p_longation, subst, Arbic_text)

    Arbic_text = Arbic_text.replace('وو', 'و')
    Arbic_text = Arbic_text.replace('يي', 'ي')
    Arbic_text = Arbic_text.replace('اا', 'ا')
    for i in range(0, len(List_search)):
        Arbic_text = Arbic_text.replace(List_search[i], List_replace[i])  
    Arbic_text = Arbic_text.strip()
    return Arbic_text

def get_vector(n_model,dim, token):
    vec = np.zeros(dim)
    is_vec = False
    if token not in n_model.wv:
        _count = 0
        is_vec = True
        for w in token.split("_"):
            if w in n_model.wv:
                _count += 1
                vec += n_model.wv[w]
        if _count > 0:
            vec = vec / _count
    else:
        vec = n_model.wv[token]
    return vec


#================Cell 4 ==================
Load the model full_grams_cbow_300_twitter.mdl and create an object from it

In [ ]:
 model_Full_grams = gensim.models.Word2Vec.load('full_grams_cbow_300_twitter.mdl')

#==========Cell 5================
In this cell, the set of data to be extracted properties is fetched and the property vectors are stored in a file of type Text and the Label is changed so that the positive equals (1) and the negative equals (0) and save this file for use in the classification process later

In [ ]:
df = pd.read_excel("Clean_Tweets-Clean_Tweets.xlsx")#import the Dataset 
j=0;
FileData=open('word2vce.txt','w')#open File with name word2vce   for stor the vector it he,
for i in df["text"]:
    token=Clean_and_Norm_Arabic_Text(i).replace(", ", "_")# call the cleaning function
    token=token.strip('[')
    token=token.strip(']')
    vec=get_vector(model_Full_grams,300,token)# call the get_vector function
    for ve in vec:# stor the vector to file 
        FileData.write(repr(ve)+',')
    if (df["lable"][j]=="positive"):#chack a label  and change 
      FileData.write(repr(1)+'\n')
    else:
      FileData.write(repr(0)+'\n')
    j=j+1
FileData.close();# close the File
      

 

#================Cell 6=================
In this cell, an object is created from the logistic regression model, which is one of the classification models, and the process of training the model is started, and display score to training data and testing data 

In [ ]:
def Func_Classification(PathFile):
  path=os.path.join(PathFile)
  data=pd.read_table(path,delimiter=',',header=None)
  X=data.drop(300,axis=1)
  Y=data[300]# this is Lable
  x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=1)
  obj_logist_Reg=LogisticRegression()#creat object form Logistic Regression 
  obj_logist_Reg.fit(x_train,y_train)#call training Function 
  print("the score to Training data is:",obj_logist_Reg.score(x_train,y_train))
  print("the score to Testing data is:",obj_logist_Reg.score(x_test,y_test))
  return obj_logist_Reg

#============Cell 7==================
Call Function To Classification 

In [ ]:
print("==============Score is:==============")
obj_logist_Reg=Func_Classification('word2vce.txt')

==============Score is:==============
the score to Training data is: 0.9345794392523364
the score to Testing data is: 0.7222222222222222


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


#=======================Cell 8================
--------------Function predict------------- 

In [ ]:
def Func_predict(Text,obj_Logistc_Reg):
    token=Clean_and_Norm_Arabic_Text(Text).replace(", ", "_")# call the cleaning function
    token=token.strip('[')
    token=token.strip(']')
    print(token)
    vec=get_vector(model_Full_grams,300,token)# call the get_vector function
    Y_Prd=obj_Logistc_Reg.predict([vec]);
    Result=["positive" if Y==1 else "negative" for Y in Y_Prd]
    return Result

#===================Cell 9=================
Call the Function predict

In [ ]:
df = pd.read_excel("Clean_Tweets-Clean_Tweets.xlsx")
print(Func_predict(df["text"][1],obj_logist_Reg))
print(Func_predict(df["text"][25],obj_logist_Reg))

اخذت_الجرعه_الثالثه_الصباح_ونمت
['positive']
اخذت_الجرعه_الثالثه_ورجعت_نمت_صحيت_احسني_مختلفه_كاني_انا_يارب_تكون_بدايه_التحول_الي_اطمح_سنيني_كلها
['negative']


In [ ]:
print("===============End================")

===============End================
